# **ROLL-RATE ANALYSIS**

In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import utils.utils as utils

# !pip freeze > ../requirements.txt

In [60]:
data = pd.read_csv(r'..\data\data_rollrate.csv')

In [61]:
# Ordering the STATUS by DPD:
status_order = ['DPD0', 'DPD1-30', 'DPD31-60', 'DPD61-90', 'DPD91-120', 'DPD120+']
data['STATUS'] = pd.Categorical(data['STATUS'], categories=status_order, ordered=True)
data = data.sort_values(['SK_ID_BUREAU', 'MONTHS_BALANCE'], ascending=True)

In [66]:
def _rollrate(data, first_month, first_obs, second_obs):
    id_obs = data.loc[data.MONTHS_BALANCE == first_month]
    merged_first = id_obs.merge(data[data['MONTHS_BALANCE'] == first_month + first_obs][['SK_ID_BUREAU', 'STATUS']],on='SK_ID_BUREAU',how='left',suffixes=('', '_first'))
    id_obs['first_interval'] = merged_first['STATUS_first']
    merged_second = id_obs.merge(data[data['MONTHS_BALANCE'] == first_month + second_obs][['SK_ID_BUREAU', 'STATUS']],on='SK_ID_BUREAU',how='left',suffixes=('', '_second'))
    id_obs['second_interval'] = merged_second['STATUS_second']
    pivot_bad = id_obs.pivot_table(index='first_interval', columns='second_interval', values='SK_ID_BUREAU', aggfunc='count').fillna(0).astype(int)
    pivot_norm = round(pivot_bad.div(pivot_bad.sum(axis=1), axis=0) * 100, 3)
    return pivot_norm

In [67]:
_rollrate(data, 36, 12, 18)

second_interval,DPD0,DPD1-30,DPD31-60,DPD61-90,DPD91-120,DPD120+
first_interval,,,,,,
DPD0,99.403,0.531,0.022,0.022,0.022,0.0
DPD1-30,10.440,89.560,0.000,0.000,0.000,0.0
DPD31-60,5.556,5.556,88.889,0.000,0.000,0.0
DPD61-90,9.091,0.000,0.000,90.909,0.000,0.0
DPD91-120,0.000,0.000,0.000,0.000,100.000,0.0
DPD120+,0.000,0.000,0.000,0.000,0.000,100.0


In [68]:
_rollrate(data, 24, 12, 18)

second_interval,DPD0,DPD1-30,DPD31-60,DPD61-90,DPD91-120,DPD120+
first_interval,,,,,,
DPD0,99.311,0.689,0.000,0.000,0.0,0.0
DPD1-30,12.088,86.813,1.099,0.000,0.0,0.0
DPD31-60,16.667,0.000,75.000,8.333,0.0,0.0
DPD61-90,0.000,0.000,0.000,75.000,0.0,25.0
DPD91-120,0.000,0.000,0.000,0.000,100.0,0.0
DPD120+,0.000,0.000,0.000,0.000,0.0,100.0


In [69]:
_rollrate(data, 1, 12, 18)

second_interval,DPD0,DPD1-30,DPD31-60,DPD61-90,DPD91-120,DPD120+
first_interval,,,,,,
DPD0,98.515,1.485,0.0,0.0,0.0,0.000
DPD1-30,20.000,80.000,0.0,0.0,0.0,0.000
DPD31-60,25.000,0.000,75.0,0.0,0.0,0.000
DPD61-90,NaN,NaN,NaN,NaN,NaN,NaN
DPD91-120,NaN,NaN,NaN,NaN,NaN,NaN
DPD120+,33.333,0.000,0.0,0.0,0.0,66.667
